# Plotting flooding data

Now we have our .tif file written out for a single country. We want to transform this data first to a shapefile. 

In the following example, you can use this code "out of the box" in order to convert your file into a shapefile. 

In [ ]:
for idx, country in countries.iterrows():
    
    if not country['iso3'] == 'RWA': # if the current country iso3 does not match RWA...
        continue                     # continue in the loop to the next country 
        
    folder = os.path.join('..', 'data', 'processed', iso3, 'hazards', 'inunriver')
    if not os.path.exists(folder):
        os.mkdir(folder)
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.shp'
    path_out = os.path.join(folder, filename)

    folder = os.path.join('..', 'data', 'processed', iso3, 'hazards', 'inunriver')
    if not os.path.exists(folder):
        os.mkdir(folder)
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif'
    path_in = os.path.join(folder, filename)

    with rasterio.open(path_in) as src:

        affine = src.transform
        array = src.read(1)

        output = []

        for vec in rasterio.features.shapes(array):

            if vec[1] > 0 and not vec[1] == 255:

                coordinates = [i for i in vec[0]['coordinates'][0]]

                coords = []

                for i in coordinates:

                    x = i[0]
                    y = i[1]

                    x2, y2 = src.transform * (x, y)

                    coords.append((x2, y2))

                output.append({
                    'type': vec[0]['type'],
                    'geometry': {
                        'type': 'Polygon',
                        'coordinates': [coords],
                    },
                    'properties': {
                        'value': vec[1],
                    }
                })

    output = geopandas.GeoDataFrame.from_features(output, crs='epsg:4326')
    output.to_file(path_out, driver='ESRI Shapefile')


Now you have your shapefile data layer in place, we can begin to plot that code.  

We will need our plotting packages.

In [ ]:
# Example
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx

First, let's set up our plotting interface.

In [ ]:
# Example
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

Set the path for our shapefile data.

In [ ]:
# Example
folder = os.path.join('..', 'data', 'processed', 'RWA', 'hazards', 'inunriver')
filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.shp'
path_in = os.path.join(folder, filename)
path_in

Load in our shapefile data.

In [ ]:
# Example
hazard = geopandas.read_file(path_in, crs='epsg:3857')
hazard = hazard.to_crs(4326)
hazard.plot(color='blue', linewidth=1.5, alpha=.7, legend=True, edgecolor=None, ax=ax)

We will want to add a `contextily` basemap.

In [ ]:
# Example
cx.add_basemap(ax, crs='epsg:4326')

And create a title for our plot.

In [ ]:
# Example
hazard_type = filename.split('_')[0]
scenario = filename.split('_')[1]
model = filename.split('_')[2]
year = filename.split('_')[3]
return_period = filename.split('_')[4]

return_period = return_period.replace('.shp', '')

main_title = 'Projected River Flooding: {}, {}, {}, {}, {}, {}'.format('RWA', hazard_type, scenario, model, year, return_period)
plt.suptitle(main_title, fontsize=14, wrap=True)

In [ ]:
Finally, we can write out our figure:

In [ ]:
# Example
path_out = os.path.join('..', 'data', 'processed', 'RWA', 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.png')

plt.savefig(path_out)
plt.close()

print('Figure processing complete')

# Exercise 1.4

Try generate a map for Grenada.

Convert the flood layer color to black. 